## AEP

In [ ]:
import pandas as pd
import numpy as np
from prophet import Prophet
import matplotlib.pyplot as plt
from torch.utils.data import Dataset, DataLoader
import torch
import torch.nn as nn
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error

%load_ext autoreload
%autoreload 2

In [ ]:
df_aep = pd.read_csv("data/AEP_hourly.csv")
df_aep.rename(columns={'Datetime': 'ds', 'AEP_MW': 'y'}, inplace=True)
df_aep['ds'] = pd.to_datetime(df_aep['ds'], format='%Y-%m-%d %H:%M:%S')
out_dir = 'AEP_results'

### test prophet based without seasonality

In [ ]:
import prophet_linear_adjust as prophet_based
daily_demand = df_aep.groupby(df_aep['ds'].dt.date)['y'].sum().reset_index()
date_start = pd.to_datetime('2004-10-01 00:00:00')
date_end = pd.to_datetime('2008-10-01 00:00:00')

results_prophet = []
i = 1

while True:

    print(f"fit model {date_end + pd.Timedelta(days=1)}")
    result = prophet_based.forecast_next_day_hourly(df_aep, date_start, date_end, daily_demand, manual = False, daily = True)
    result['h_ahead'] = pd.to_datetime(result['ds']).dt.hour
    results_prophet.append(result)

    date_start += pd.Timedelta(days=1)
    date_end += pd.Timedelta(days=1)
    if i % 5 == 0:
        results_prophet.to_csv('experiment_results/' + out_dir + '/results_prophet_dwy.csv', index=False)
    if i == 70:
        break
    else:
        i += 1

results_prophet = pd.concat(results_prophet, ignore_index=True)

results_prophet.to_csv('experiment_results/' + out_dir + '/results_prophet.csv', index=False)

### test prophet based with daily, weekly, yearly seasonality

In [ ]:
daily_demand = df_aep.groupby(df_aep['ds'].dt.date)['y'].sum().reset_index()
date_start = pd.to_datetime('2004-10-01 00:00:00')
date_end = pd.to_datetime('2008-10-01 00:00:00')

results_prophet_dwy = []
i = 1

while True:

    print(f"fit model {date_end + pd.Timedelta(days=1)}")
    result = prophet_based.forecast_next_day_hourly(df_aep, date_start, date_end, daily_demand, manual = False, daily = True, weekly = True, yearly = True)
    result['h_ahead'] = pd.to_datetime(result['ds']).dt.hour
    results_prophet_dwy.append(result)

    date_start += pd.Timedelta(days=1)
    date_end += pd.Timedelta(days=1)
    if i % 5 == 0:
        results_prophet_dwy.to_csv('experiment_results/' + out_dir + '/results_prophet_dwy.csv', index=False)
    if i == 70:
        break
    else:
        i += 1

results_prophet_dwy = pd.concat(results_prophet_dwy, ignore_index=True)

results_prophet_dwy.to_csv('experiment_results/' + out_dir + '/results_prophet_dwy.csv', index=False)

### test Simple RNN

In [ ]:
import RNN_simple
train_config = RNN_simple.training_config(n_epochs = 30, device = torch.device("mps"))
results_simple = pd.DataFrame({'day': [], 'hour': [], 'y_hat': [], 'y':[]})
date_start = pd.to_datetime('2004-10-01 00:00:00')
date_end = pd.to_datetime('2008-10-01 00:00:00')

i = 1

while True:
    print(i)
    df_reduced = df_aep.loc[(df_aep['ds'] >= date_start) & (df_aep['ds'] < date_end)].copy()
    model = RNN_simple.RNN_simple(latent_dim=32, activation='relu')
    trainer = RNN_simple.RNN_train_simple(model, train_config)
    forcast, true = trainer(df_reduced)

    result = pd.DataFrame({'day': df_reduced['ds'].dt.date.max(), 'hour': list(range(24)), 'y_hat': forcast, 'y': true})

    results_simple = pd.concat([results_simple, result]).reset_index(drop=True)

    date_start += pd.Timedelta(days=1)
    date_end += pd.Timedelta(days=1)
    if i % 5 == 0:
        results_simple.to_csv('experiment_results/' + out_dir + '/results_simple.csv', index=False)
    if i == 70:
        print('finish')
        break
    else:
        i += 1

results_simple.to_csv('experiment_results/' + out_dir + '/results_simple.csv', index=False)

### test RNN+self convolution

In [ ]:
import RNN_attention
train_config = RNN_attention.training_config(n_epochs = 30, device = torch.device("mps"))
results_attention = pd.DataFrame({'day': [], 'hour': [], 'y_hat': [], 'y':[]})
date_start = pd.to_datetime('2004-10-01 00:00:00')
date_end = pd.to_datetime('2008-10-01 00:00:00')

i = 1

while True:
    print(i)
    df_reduced = df_aep.loc[(df_aep['ds'] >= date_start) & (df_aep['ds'] < date_end)].copy()
    model = RNN_attention.RNN_attention(latent_dim=32, d_model=128, nhead=4)
    trainer = RNN_attention.RNN_train_attention(model, train_config)
    forcast, true = trainer(df_reduced)
    
    result = pd.DataFrame({'day': df_reduced['ds'].dt.date.max(), 'hour': list(range(24)), 'y_hat': forcast, 'y': true})

    results_attention = pd.concat([results_attention, result]).reset_index(drop=True)

    date_start += pd.Timedelta(days=1)
    date_end += pd.Timedelta(days=1)
    if i % 5 == 0:
        results_attention.to_csv('experiment_results/' + out_dir + '/results_attention.csv', index=False)
    if i == 70:
        print('finish')
        break
    else:
        i += 1

results_attention.to_csv('experiment_results/' + out_dir + '/results_attention.csv', index=False)

### test RNN+self convolution + fourier matrix (weekly, monthly, yearly)

In [ ]:
import RNN_fourier_RNN

train_config = RNN_fourier_RNN.training_config(n_epochs=30, device=torch.device("mps"))
fourier_conf = RNN_fourier_RNN.fourier_config(mode="matrix", K_weekly=3, K_monthly=6, K_yearly=10)

# Calculate dimensions correctly
K_total = fourier_conf.K_weekly + fourier_conf.K_monthly + fourier_conf.K_yearly
F_per_hour = 2 * K_total  # sin/cos pairs per hour

if fourier_conf.mode == "vector":
    cont_dim = 1 + F_per_hour
    fourier_dim = F_per_hour
else:  # matrix mode
    cont_dim = 1 + 24 * F_per_hour  # Each hour gets F_per_hour features
    fourier_dim = F_per_hour

results_fourier = pd.DataFrame({'day': [], 'hour': [], 'y_hat': [], 'y':[]})
date_start = pd.to_datetime('2004-10-01 00:00:00')
date_end = pd.to_datetime('2008-10-01 00:00:00')

i = 1

while True:
    print(i)
    df_reduced = df_aep.loc[(df_aep['ds'] >= date_start) & (df_aep['ds'] < date_end)].copy()
    model = RNN_fourier_RNN.RNN_fourier(cont_dim=cont_dim, fourier_dim=fourier_dim, xf_mode="matrix",
                                               d_model=128,latent_dim=32,nhead=4)
    trainer = RNN_fourier_RNN.RNN_train_fourier(model, train_config, fourier_conf)
    forcast, true = trainer(df_reduced)

    result = pd.DataFrame({'day': df_reduced['ds'].dt.date.max(), 'hour': list(range(24)), 'y_hat': forcast, 'y': true})

    results_fourier = pd.concat([results_fourier, result]).reset_index(drop=True)

    date_start += pd.Timedelta(days=1)
    date_end += pd.Timedelta(days=1)
    if i % 5 == 0:
        results_fourier.to_csv('experiment_results/' + out_dir + '/results_fourier.csv', index=False)
    if i == 70:
        print('finish')
        break
    else:
        i += 1

results_fourier.to_csv('experiment_results/' + out_dir + '/results_fourier.csv', index=False)

In [ ]:
k = 24*3
x_ax = np.arange(k)

plt.plot(x_ax, results_fourier['y_hat'][:k], marker="o", linestyle="-", color="b")
plt.plot(x_ax, results_fourier['y'][:k], marker="o", linestyle="-", color="r")
plt.xlabel("Index")
plt.ylabel("y_Tplus1")
plt.title("Line plot of y_Tplus1")
plt.grid(True)
plt.show()

## Comed

In [ ]:
import pandas as pd
import numpy as np
from prophet import Prophet
import matplotlib.pyplot as plt
from torch.utils.data import Dataset, DataLoader
import torch
import torch.nn as nn
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error
import model_summary as model_class
import prophet_linear_adjust as prophet_based

%load_ext autoreload
%autoreload 2

In [ ]:
df_comed = pd.read_csv("COMED_hourly.csv")
df_comed.rename(columns={'Datetime': 'ds', 'COMED_MW': 'y'}, inplace=True)
df_comed['ds'] = pd.to_datetime(df_comed['ds'], format='%Y-%m-%d %H:%M:%S')

out_dir = 'Comed_results'

### test prophet based without seasonality

In [ ]:
daily_demand_comed = df_comed.groupby(df_comed['ds'].dt.date)['y'].sum().reset_index()
date_start = pd.to_datetime('2012-10-01 00:00:00')
date_end = pd.to_datetime('2016-10-01 00:00:00')

results_prophet = []
i = 1

while True:

    print(f"fit model {date_end + pd.Timedelta(days=1)}")
    result = prophet_based.forecast_next_day_hourly(df_comed, date_start, date_end, daily_demand_comed, manual = False, daily = True)
    result['h_ahead'] = pd.to_datetime(result['ds']).dt.hour
    results_prophet.append(result)

    date_start += pd.Timedelta(days=1)
    date_end += pd.Timedelta(days=1)

    if i == 70:
        break
    else:
        i += 1

results_prophet = pd.concat(results_prophet, ignore_index=True)

results_prophet.to_csv(out_dir + '/results_prophet.csv', index=False)

### test prophet based with weekly seasonality

In [ ]:
daily_demand_comed = df_comed.groupby(df_comed['ds'].dt.date)['y'].sum().reset_index()
date_start = pd.to_datetime('2012-10-01 00:00:00')
date_end = pd.to_datetime('2016-10-01 00:00:00')

results_prophet_w = []
i = 1

while True:

    print(f"fit model {date_end + pd.Timedelta(days=1)}")
    result = prophet_based.forecast_next_day_hourly(df_comed, date_start, date_end, daily_demand_comed, manual = False, daily = True, weekly = True)
    result['h_ahead'] = pd.to_datetime(result['ds']).dt.hour
    results_prophet_w.append(result)

    date_start += pd.Timedelta(days=1)
    date_end += pd.Timedelta(days=1)

    if i == 70:
        break
    else:
        i += 1

results_prophet_w = pd.concat(results_prophet_w, ignore_index=True)

results_prophet_w.to_csv(out_dir + '/results_prophet_w.csv', index=False)

### test RNN+self convolution

In [ ]:
train_config = model_class.training_config(n_epochs = 30, device = torch.device("mps"))
results_aep_self_con = pd.DataFrame({'day': [], 'hour': [], 'y_hat': [], 'y':[]})
date_start = pd.to_datetime('2012-10-01 00:00:00')
date_end = pd.to_datetime('2016-10-01 00:00:00')

i = 1

while True:
    print(i)
    df_reduced = df_comed.loc[(df_comed['ds'] >= date_start) & (df_comed['ds'] < date_end)].copy()
    model = model_class.RNN_FeatureAttention()
    trainer = model_class.RNN_train_1(model, train_config)
    forcast, true = trainer(df_reduced)
    
    result = pd.DataFrame({'day': df_reduced['ds'].dt.date.max(), 'hour': list(range(24)), 'y_hat': forcast, 'y': true})

    results_aep_self_con = pd.concat([results_aep_self_con, result]).reset_index(drop=True)

    date_start += pd.Timedelta(days=1)
    date_end += pd.Timedelta(days=1)
    if i == 70:
        print('finish')
        break
    else:
        i += 1

results_aep_self_con.to_csv(out_dir + '/results_aep_self_con.csv', index=False)

### test RNN+self convolution + numerical weekday

In [ ]:
train_config = model_class.training_config(n_epochs = 30, device = torch.device("mps"))
results_aep_self_con_wn = pd.DataFrame({'day': [], 'hour': [], 'y_hat': [], 'y':[]})
date_start = pd.to_datetime('2012-10-01 00:00:00')
date_end = pd.to_datetime('2016-10-01 00:00:00')

i = 1

while True:
    print(i)
    df_reduced = df_comed.loc[(df_comed['ds'] >= date_start) & (df_comed['ds'] < date_end)].copy()
    model = model_class.RNN_FeatureAttention_wn()
    trainer = model_class.RNN_train_1(model, train_config)
    forcast, true = trainer(df_reduced)

    result = pd.DataFrame({'day': df_reduced['ds'].dt.date.max(), 'hour': list(range(24)), 'y_hat': forcast, 'y': true})

    results_aep_self_con_wn = pd.concat([results_aep_self_con_wn, result]).reset_index(drop=True)

    date_start += pd.Timedelta(days=1)
    date_end += pd.Timedelta(days=1)
    if i == 70:
        print('finish')
        break
    else:
        i += 1

results_aep_self_con_wn.to_csv(out_dir + '/results_aep_self_con_wn.csv', index=False)

### test RNN+self convolution + categorical weekday

In [ ]:
train_config = model_class.training_config(n_epochs = 30, device = torch.device("mps"))
results_aep_self_con_wc = pd.DataFrame({'day': [], 'hour': [], 'y_hat': [], 'y':[]})
date_start = pd.to_datetime('2012-10-01 00:00:00')
date_end = pd.to_datetime('2016-10-01 00:00:00')

i = 1

while True:
    print(i)
    df_reduced = df_comed.loc[(df_comed['ds'] >= date_start) & (df_comed['ds'] < date_end)].copy()
    model = model_class.RNN_FeatureAttention_wc()
    trainer = model_class.RNN_train_2(model, train_config)
    forcast, true = trainer(df_reduced)
    
    result = pd.DataFrame({'day': df_reduced['ds'].dt.date.max(), 'hour': list(range(24)), 'y_hat': forcast, 'y': true})

    results_aep_self_con_wc = pd.concat([results_aep_self_con_wc, result]).reset_index(drop=True)

    date_start += pd.Timedelta(days=1)
    date_end += pd.Timedelta(days=1)
    if i == 70:
        print('finish')
        break
    else:
        i += 1
results_aep_self_con_wc.to_csv(out_dir + '/results_aep_self_con_wc.csv', index=False)

### test RNN+self convolution + fourier vector (only weekly)

In [ ]:
train_config = model_class.training_config(n_epochs = 30, device = torch.device("mps"))
fourier_conf = model_class.fourier_config(mode="vector", K_weekly=3, K_monthly=0)
F_pairs = 2*(fourier_conf.K_weekly + fourier_conf.K_monthly)
if fourier_conf.mode == "vector":
    cont_dim = 1 + F_pairs
    fourier_dim = F_pairs
else:
    cont_dim = 1 + 24*F_pairs
    fourier_dim = F_pairs
            
results_aep_fourier_w = pd.DataFrame({'day': [], 'hour': [], 'y_hat': [], 'y':[]})
date_start = pd.to_datetime('2012-10-01 00:00:00')
date_end = pd.to_datetime('2016-10-01 00:00:00')

i = 1

while True:
    print(i)
    df_reduced = df_comed.loc[(df_comed['ds'] >= date_start) & (df_comed['ds'] < date_end)].copy()
    model = model_class.RNN_fourier(cont_dim=cont_dim, fourier_dim=fourier_dim,
                                    xf_mode=fourier_conf.mode, d_model=128, nhead=4, 
                                    activation="relu", learn_z0=True, dropout=0.0,
                                    H=24, use_gate=True, nonneg_U0=False)
    trainer = model_class.RNN_train_fourier(model, train_config, fourier_conf)
    forcast, true = trainer(df_reduced)
    
    result = pd.DataFrame({'day': df_reduced['ds'].dt.date.max(), 'hour': list(range(24)), 'y_hat': forcast, 'y': true})

    results_aep_fourier_w = pd.concat([results_aep_fourier_w, result]).reset_index(drop=True)

    date_start += pd.Timedelta(days=1)
    date_end += pd.Timedelta(days=1)
    if i == 70:
        print('finish')
        break
    else:
        i += 1

results_aep_fourier_w.to_csv(out_dir + '/results_aep_fourier_w.csv', index=False)

### test RNN+self convolution + fourier matrix (only weekly)

In [ ]:
train_config = model_class.training_config(n_epochs = 30, device = torch.device("mps"))
fourier_conf = model_class.fourier_config(mode="matrix", K_weekly=3, K_monthly=0)
F_pairs = 2*(fourier_conf.K_weekly + fourier_conf.K_monthly)
if fourier_conf.mode == "vector":
    cont_dim = 1 + F_pairs
    fourier_dim = F_pairs
else:
    cont_dim = 1 + 24*F_pairs
    fourier_dim = F_pairs
            
results_aep_fourier_w_matrix = pd.DataFrame({'day': [], 'hour': [], 'y_hat': [], 'y':[]})
date_start = pd.to_datetime('2012-10-01 00:00:00')
date_end = pd.to_datetime('2016-10-01 00:00:00')

i = 1

while True:
    print(i)
    df_reduced = df_comed.loc[(df_comed['ds'] >= date_start) & (df_comed['ds'] < date_end)].copy()
    model = model_class.RNN_fourier(cont_dim=cont_dim, fourier_dim=fourier_dim,
                                    xf_mode=fourier_conf.mode, d_model=128, nhead=4, 
                                    activation="relu", learn_z0=True, dropout=0.0,
                                    H=24, use_gate=True, nonneg_U0=False)
    trainer = model_class.RNN_train_fourier(model, train_config, fourier_conf)
    forcast, true = trainer(df_reduced)
    
    result = pd.DataFrame({'day': df_reduced['ds'].dt.date.max(), 'hour': list(range(24)), 'y_hat': forcast, 'y': true})

    results_aep_fourier_w_matrix = pd.concat([results_aep_fourier_w_matrix, result]).reset_index(drop=True)

    date_start += pd.Timedelta(days=1)
    date_end += pd.Timedelta(days=1)
    if i == 70:
        print('finish')
        break
    else:
        i += 1

results_aep_fourier_w_matrix.to_csv(out_dir + '/results_aep_fourier_w_matrix.csv', index=False)

In [ ]:
train_config = model_class.training_config(n_epochs = 30, device = torch.device("mps"))
fourier_conf = model_class.fourier_config(mode="matrix", K_weekly=3, K_monthly=6, K_yearly=10)
F_pairs = 2*(fourier_conf.K_weekly + fourier_conf.K_monthly + fourier_conf.K_yearly)
if fourier_conf.mode == "vector":
    cont_dim = 1 + F_pairs
    fourier_dim = F_pairs
else:
    cont_dim = 1 + 24*F_pairs
    fourier_dim = F_pairs

results_aep_fourier_w_matrix = pd.DataFrame({'day': [], 'hour': [], 'y_hat': [], 'y':[]})
date_start = pd.to_datetime('2012-10-01 00:00:00')
date_end = pd.to_datetime('2016-10-01 00:00:00')

i = 1

while True:
    print(i)
    df_reduced = df_comed.loc[(df_comed['ds'] >= date_start) & (df_comed['ds'] < date_end)].copy()
    model = model_class.RNN_fourier(cont_dim=cont_dim, fourier_dim=fourier_dim,
                                    xf_mode=fourier_conf.mode, d_model=128, nhead=4,
                                    activation="relu", learn_z0=True, dropout=0.0,
                                    H=24, use_gate=True, nonneg_U0=False)
    trainer = model_class.RNN_train_fourier(model, train_config, fourier_conf)
    forcast, true = trainer(df_reduced)

    result = pd.DataFrame({'day': df_reduced['ds'].dt.date.max(), 'hour': list(range(24)), 'y_hat': forcast, 'y': true})

    results_aep_fourier_w_matrix = pd.concat([results_aep_fourier_w_matrix, result]).reset_index(drop=True)

    date_start += pd.Timedelta(days=1)
    date_end += pd.Timedelta(days=1)
    if i == 70:
        print('finish')
        break
    else:
        i += 1

results_aep_fourier_w_matrix.to_csv(out_dir + '/results_aep_fourier_kw3_km6_ky10_matrix.csv', index=False)

### test RNN_based Asym

In [ ]:
train_config = model_class.training_config(n_epochs = 30, device = torch.device("mps"))
results_aep_asy_con = pd.DataFrame({'day': [], 'hour': [], 'y_hat': [], 'y':[]})
date_start = pd.to_datetime('2012-10-01 00:00:00')
date_end = pd.to_datetime('2016-10-01 00:00:00')

i = 1

while True:
    print(i)
    df_reduced = df_comed.loc[(df_comed['ds'] >= date_start) & (df_comed['ds'] < date_end)].copy()
    model = model_class.RNN_asym_feature()
    trainer = model_class.RNN_train_1(model, train_config)
    forcast, true = trainer(df_reduced)

    result = pd.DataFrame({'day': df_reduced['ds'].dt.date.max(), 'hour': list(range(24)), 'y_hat': forcast, 'y': true})

    results_aep_asy_con = pd.concat([results_aep_asy_con, result]).reset_index(drop=True)

    date_start += pd.Timedelta(days=1)
    date_end += pd.Timedelta(days=1)
    if i == 70:
        print('finish')
        break
    else:
        i += 1

results_aep_asy_con.to_csv(out_dir + '/results_aep_asy_con.csv', index=False)

In [ ]:
out_dir

### Test RNN_based ARMA

In [ ]:
train_config = model_class.training_config(n_epochs = 30, device = torch.device("cpu"))
results_arma_asy_con = pd.DataFrame({'day': [], 'hour': [], 'y_hat': [], 'y':[]})
date_start = pd.to_datetime('2012-10-01 00:00:00')
date_end = pd.to_datetime('2016-10-01 00:00:00')

i = 1

while True:
    print(i)
    df_reduced = df_comed.loc[(df_comed['ds'] >= date_start) & (df_comed['ds'] < date_end)].copy()
    model = model_class.RNN_ARMA_Feature()
    trainer = model_class.RNN_train_1(model, train_config)
    forcast, true = trainer(df_reduced)
    result = pd.DataFrame({'day': df_reduced['ds'].dt.date.max(), 'hour': list(range(24)), 'y_hat': forcast, 'y': true})

    results_arma_asy_con = pd.concat([results_arma_asy_con, result]).reset_index(drop=True)

    date_start += pd.Timedelta(days=1)
    date_end += pd.Timedelta(days=1)
    if i == 70:
        print('finish')
        break
    else:
        i += 1

results_arma_asy_con.to_csv(out_dir + '/results_arma_asy_con.csv', index=False)

### test Dirich

In [ ]:
import model_dich

class Cfg: pass
cfg = Cfg()
cfg.device = "mps"
cfg.n_epochs = 90
cfg.lr = 5e-4
cfg.x_col = "x"
cfg.y_cols = [f"y_{i}" for i in range(24)]
cfg.T_hist = 32
cfg.batch_size = 64
cfg.test_ratio = 0.2
cfg.kl_coeff = 1.0


train_config = model_class.training_config(n_epochs = 3, device = torch.device("cpu"))
results_prob_dich = pd.DataFrame({'day': [], 'hour': [], 'y_hat': [], 'y':[]})
date_start = pd.to_datetime('2012-10-01 00:00:00')
date_end = pd.to_datetime('2016-10-01 00:00:00')

i = 1

while True:
    print(i)
    df_reduced = df_comed.loc[(df_comed['ds'] >= date_start) & (df_comed['ds'] < date_end)].copy()
    model = model_dich.DirichletComposerRNN(device=torch.device("cpu"))
    trainer = model_dich.RNN_train(model, cfg)
    ret = trainer(df_reduced)
    result = pd.DataFrame({'day': df_reduced['ds'].dt.date.max(), 'hour': list(range(24)), 'y_hat': ret['Y_pred'].flatten(), 'y': ret['Y_test'].flatten()})

    results_prob_dich = pd.concat([results_prob_dich, result]).reset_index(drop=True)

    date_start += pd.Timedelta(days=1)
    date_end += pd.Timedelta(days=1)
    if i == 70:
        print('finish')
        break
    else:
        i += 1

results_prob_dich.to_csv(out_dir + '/results_prob_dich_90epoch.csv', index=False)

In [ ]:
out_dir

## DAYTON

In [ ]:
import pandas as pd
import numpy as np
from prophet import Prophet
import matplotlib.pyplot as plt
from torch.utils.data import Dataset, DataLoader
import torch
import torch.nn as nn
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error
import model_summary as model_class
import prophet_linear_adjust as prophet_based

%load_ext autoreload
%autoreload 2

In [ ]:
df_dayton = pd.read_csv("DAYTON_hourly.csv")
df_dayton.rename(columns={'Datetime': 'ds', 'DAYTON_MW': 'y'}, inplace=True)
df_dayton['ds'] = pd.to_datetime(df_dayton['ds'], format='%Y-%m-%d %H:%M:%S')
out_dir = 'Dayton_results'

### test prophet based without seasonality

In [ ]:
daily_demand_dayton = df_dayton.groupby(df_dayton['ds'].dt.date)['y'].sum().reset_index()
date_start = pd.to_datetime('2012-10-01 00:00:00')
date_end = pd.to_datetime('2016-10-01 00:00:00')

results_prophet = []
i = 1

while True:

    print(f"fit model {date_end + pd.Timedelta(days=1)}")
    result = prophet_based.forecast_next_day_hourly(df_dayton, date_start, date_end, daily_demand_dayton, manual = False, daily = True)
    result['h_ahead'] = pd.to_datetime(result['ds']).dt.hour
    results_prophet.append(result)

    date_start += pd.Timedelta(days=1)
    date_end += pd.Timedelta(days=1)

    if i == 70:
        break
    else:
        i += 1

results_prophet = pd.concat(results_prophet, ignore_index=True)

results_prophet.to_csv(out_dir + '/results_prophet.csv', index=False)

### test prophet based with seasonality

In [ ]:
daily_demand_dayton = df_dayton.groupby(df_dayton['ds'].dt.date)['y'].sum().reset_index()
date_start = pd.to_datetime('2012-10-01 00:00:00')
date_end = pd.to_datetime('2016-10-01 00:00:00')

results_prophet_w = []
i = 1

while True:

    print(f"fit model {date_end + pd.Timedelta(days=1)}")
    result = prophet_based.forecast_next_day_hourly(df_dayton, date_start, date_end, daily_demand_dayton, manual = False, daily = True, weekly = True)
    result['h_ahead'] = pd.to_datetime(result['ds']).dt.hour
    results_prophet_w.append(result)

    date_start += pd.Timedelta(days=1)
    date_end += pd.Timedelta(days=1)

    if i == 70:
        break
    else:
        i += 1

results_prophet_w = pd.concat(results_prophet_w, ignore_index=True)

results_prophet_w.to_csv(out_dir + '/results_prophet_w.csv', index=False)

### test RNN+self convolution

In [ ]:
train_config = model_class.training_config(n_epochs = 30, device = torch.device("mps"))
results_aep_self_con = pd.DataFrame({'day': [], 'hour': [], 'y_hat': [], 'y':[]})
date_start = pd.to_datetime('2012-10-01 00:00:00')
date_end = pd.to_datetime('2016-10-01 00:00:00')

i = 1

while True:
    print(i)
    df_reduced = df_dayton.loc[(df_dayton['ds'] >= date_start) & (df_dayton['ds'] < date_end)].copy()
    model = model_class.RNN_FeatureAttention()
    trainer = model_class.RNN_train_1(model, train_config)
    forcast, true = trainer(df_reduced)
    
    result = pd.DataFrame({'day': df_reduced['ds'].dt.date.max(), 'hour': list(range(24)), 'y_hat': forcast, 'y': true})

    results_aep_self_con = pd.concat([results_aep_self_con, result]).reset_index(drop=True)

    date_start += pd.Timedelta(days=1)
    date_end += pd.Timedelta(days=1)
    if i == 70:
        print('finish')
        break
    else:
        i += 1

results_aep_self_con.to_csv(out_dir + '/results_aep_self_con.csv', index=False)

### test RNN+self convolution + numerical weekday

In [ ]:
train_config = model_class.training_config(n_epochs = 30, device = torch.device("mps"))
results_aep_self_con_wn = pd.DataFrame({'day': [], 'hour': [], 'y_hat': [], 'y':[]})
date_start = pd.to_datetime('2012-10-01 00:00:00')
date_end = pd.to_datetime('2016-10-01 00:00:00')

i = 1

while True:
    print(i)
    df_reduced = df_dayton.loc[(df_dayton['ds'] >= date_start) & (df_dayton['ds'] < date_end)].copy()
    model = model_class.RNN_FeatureAttention_wn()
    trainer = model_class.RNN_train_1(model, train_config)
    forcast, true = trainer(df_reduced)
    
    result = pd.DataFrame({'day': df_reduced['ds'].dt.date.max(), 'hour': list(range(24)), 'y_hat': forcast, 'y': true})

    results_aep_self_con_wn = pd.concat([results_aep_self_con_wn, result]).reset_index(drop=True)

    date_start += pd.Timedelta(days=1)
    date_end += pd.Timedelta(days=1)
    if i == 70:
        print('finish')
        break
    else:
        i += 1

results_aep_self_con_wn.to_csv(out_dir + '/results_aep_self_con_wn.csv', index=False)

### test RNN+self convolution + categorical weekday

In [ ]:
train_config = model_class.training_config(n_epochs = 30, device = torch.device("mps"))
results_aep_self_con_wc = pd.DataFrame({'day': [], 'hour': [], 'y_hat': [], 'y':[]})
date_start = pd.to_datetime('2012-10-01 00:00:00')
date_end = pd.to_datetime('2016-10-01 00:00:00')

i = 1

while True:
    print(i)
    df_reduced = df_dayton.loc[(df_dayton['ds'] >= date_start) & (df_dayton['ds'] < date_end)].copy()
    model = model_class.RNN_FeatureAttention_wc()
    trainer = model_class.RNN_train_2(model, train_config)
    forcast, true = trainer(df_reduced)
    
    result = pd.DataFrame({'day': df_reduced['ds'].dt.date.max(), 'hour': list(range(24)), 'y_hat': forcast, 'y': true})

    results_aep_self_con_wc = pd.concat([results_aep_self_con_wc, result]).reset_index(drop=True)

    date_start += pd.Timedelta(days=1)
    date_end += pd.Timedelta(days=1)
    if i == 70:
        print('finish')
        break
    else:
        i += 1
results_aep_self_con_wc.to_csv(out_dir + '/results_aep_self_con_wc.csv', index=False)

### test RNN+self convolution + fourier vector (only weekly)

In [ ]:
train_config = model_class.training_config(n_epochs = 30, device = torch.device("mps"))
fourier_conf = model_class.fourier_config(mode="vector", K_weekly=3, K_monthly=0)
F_pairs = 2*(fourier_conf.K_weekly + fourier_conf.K_monthly)
if fourier_conf.mode == "vector":
    cont_dim = 1 + F_pairs
    fourier_dim = F_pairs
else:
    cont_dim = 1 + 24*F_pairs
    fourier_dim = F_pairs
            
results_aep_fourier_w = pd.DataFrame({'day': [], 'hour': [], 'y_hat': [], 'y':[]})
date_start = pd.to_datetime('2012-10-01 00:00:00')
date_end = pd.to_datetime('2016-10-01 00:00:00')

i = 1

while True:
    print(i)
    df_reduced = df_dayton.loc[(df_dayton['ds'] >= date_start) & (df_dayton['ds'] < date_end)].copy()
    model = model_class.RNN_fourier(cont_dim=cont_dim, fourier_dim=fourier_dim,
                                    xf_mode=fourier_conf.mode, d_model=128, nhead=4, 
                                    activation="relu", learn_z0=True, dropout=0.0,
                                    H=24, use_gate=True, nonneg_U0=False)
    trainer = model_class.RNN_train_fourier(model, train_config, fourier_conf)
    forcast, true = trainer(df_reduced)
    
    result = pd.DataFrame({'day': df_reduced['ds'].dt.date.max(), 'hour': list(range(24)), 'y_hat': forcast, 'y': true})

    results_aep_fourier_w = pd.concat([results_aep_fourier_w, result]).reset_index(drop=True)

    date_start += pd.Timedelta(days=1)
    date_end += pd.Timedelta(days=1)
    if i == 70:
        print('finish')
        break
    else:
        i += 1

results_aep_fourier_w.to_csv(out_dir + '/results_aep_fourier_w.csv', index=False)

### test RNN+self convolution + fourier matrix (only weekly)

In [ ]:
train_config = model_class.training_config(n_epochs = 30, device = torch.device("mps"))
fourier_conf = model_class.fourier_config(mode="matrix", K_weekly=3, K_monthly=0)
F_pairs = 2*(fourier_conf.K_weekly + fourier_conf.K_monthly)
if fourier_conf.mode == "vector":
    cont_dim = 1 + F_pairs
    fourier_dim = F_pairs
else:
    cont_dim = 1 + 24*F_pairs
    fourier_dim = F_pairs
            
results_aep_fourier_w_matrix = pd.DataFrame({'day': [], 'hour': [], 'y_hat': [], 'y':[]})
date_start = pd.to_datetime('2012-10-01 00:00:00')
date_end = pd.to_datetime('2016-10-01 00:00:00')

i = 1

while True:
    print(i)
    df_reduced = df_dayton.loc[(df_dayton['ds'] >= date_start) & (df_dayton['ds'] < date_end)].copy()
    model = model_class.RNN_fourier(cont_dim=cont_dim, fourier_dim=fourier_dim,
                                    xf_mode=fourier_conf.mode, d_model=128, nhead=4, 
                                    activation="relu", learn_z0=True, dropout=0.0,
                                    H=24, use_gate=True, nonneg_U0=False)
    trainer = model_class.RNN_train_fourier(model, train_config, fourier_conf)
    forcast, true = trainer(df_reduced)
    
    result = pd.DataFrame({'day': df_reduced['ds'].dt.date.max(), 'hour': list(range(24)), 'y_hat': forcast, 'y': true})

    results_aep_fourier_w_matrix = pd.concat([results_aep_fourier_w_matrix, result]).reset_index(drop=True)

    date_start += pd.Timedelta(days=1)
    date_end += pd.Timedelta(days=1)
    if i == 70:
        print('finish')
        break
    else:
        i += 1

results_aep_fourier_w_matrix.to_csv(out_dir + '/results_aep_fourier_w_matrix.csv', index=False)

In [ ]:
train_config = model_class.training_config(n_epochs = 30, device = torch.device("mps"))
fourier_conf = model_class.fourier_config(mode="matrix", K_weekly=3, K_monthly=6, K_yearly=10)
F_pairs = 2*(fourier_conf.K_weekly + fourier_conf.K_monthly + fourier_conf.K_yearly)
if fourier_conf.mode == "vector":
    cont_dim = 1 + F_pairs
    fourier_dim = F_pairs
else:
    cont_dim = 1 + 24*F_pairs
    fourier_dim = F_pairs

results_aep_fourier_w_matrix = pd.DataFrame({'day': [], 'hour': [], 'y_hat': [], 'y':[]})
date_start = pd.to_datetime('2012-10-01 00:00:00')
date_end = pd.to_datetime('2016-10-01 00:00:00')

i = 1

while True:
    print(i)
    df_reduced = df_dayton.loc[(df_dayton['ds'] >= date_start) & (df_dayton['ds'] < date_end)].copy()
    model = model_class.RNN_fourier(cont_dim=cont_dim, fourier_dim=fourier_dim,
                                    xf_mode=fourier_conf.mode, d_model=128, nhead=4,
                                    activation="relu", learn_z0=True, dropout=0.0,
                                    H=24, use_gate=True, nonneg_U0=False)
    trainer = model_class.RNN_train_fourier(model, train_config, fourier_conf)
    forcast, true = trainer(df_reduced)

    result = pd.DataFrame({'day': df_reduced['ds'].dt.date.max(), 'hour': list(range(24)), 'y_hat': forcast, 'y': true})

    results_aep_fourier_w_matrix = pd.concat([results_aep_fourier_w_matrix, result]).reset_index(drop=True)

    date_start += pd.Timedelta(days=1)
    date_end += pd.Timedelta(days=1)
    if i == 70:
        print('finish')
        break
    else:
        i += 1

results_aep_fourier_w_matrix.to_csv(out_dir + '/results_aep_fourier_kw3_km6_ky10_matrix.csv', index=False)

### test RNN_based asym

In [ ]:
train_config = model_class.training_config(n_epochs = 30, device = torch.device("mps"))
results_aep_asy_con = pd.DataFrame({'day': [], 'hour': [], 'y_hat': [], 'y':[]})
date_start = pd.to_datetime('2012-10-01 00:00:00')
date_end = pd.to_datetime('2016-10-01 00:00:00')

i = 1

while True:
    print(i)
    df_reduced = df_dayton.loc[(df_dayton['ds'] >= date_start) & (df_dayton['ds'] < date_end)].copy()
    model = model_class.RNN_asym_feature()
    trainer = model_class.RNN_train_1(model, train_config)
    forcast, true = trainer(df_reduced)

    result = pd.DataFrame({'day': df_reduced['ds'].dt.date.max(), 'hour': list(range(24)), 'y_hat': forcast, 'y': true})

    results_aep_asy_con = pd.concat([results_aep_asy_con, result]).reset_index(drop=True)

    date_start += pd.Timedelta(days=1)
    date_end += pd.Timedelta(days=1)
    if i == 70:
        print('finish')
        break
    else:
        i += 1

results_aep_asy_con.to_csv(out_dir + '/results_aep_asy_con.csv', index=False)

### test RNN_based ARMA

In [ ]:
train_config = model_class.training_config(n_epochs = 30, device = torch.device("cpu"))
results_arma_asy_con = pd.DataFrame({'day': [], 'hour': [], 'y_hat': [], 'y':[]})
date_start = pd.to_datetime('2012-10-01 00:00:00')
date_end = pd.to_datetime('2016-10-01 00:00:00')

i = 1

while True:
    print(i)
    df_reduced = df_dayton.loc[(df_dayton['ds'] >= date_start) & (df_dayton['ds'] < date_end)].copy()
    model = model_class.RNN_ARMA_Feature()
    trainer = model_class.RNN_train_1(model, train_config)
    forcast, true = trainer(df_reduced)
    result = pd.DataFrame({'day': df_reduced['ds'].dt.date.max(), 'hour': list(range(24)), 'y_hat': forcast, 'y': true})

    results_arma_asy_con = pd.concat([results_arma_asy_con, result]).reset_index(drop=True)

    date_start += pd.Timedelta(days=1)
    date_end += pd.Timedelta(days=1)
    if i == 70:
        print('finish')
        break
    else:
        i += 1

results_arma_asy_con.to_csv(out_dir + '/results_arma_asy_con.csv', index=False)

## DEOK

In [ ]:
import pandas as pd
import numpy as np
from prophet import Prophet
import matplotlib.pyplot as plt
from torch.utils.data import Dataset, DataLoader
import torch
import torch.nn as nn
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error
import model_summary as model_class
import prophet_linear_adjust as prophet_based

%load_ext autoreload
%autoreload 2

In [ ]:
df_deok = pd.read_csv("DEOK_hourly.csv")
df_deok.rename(columns={'Datetime': 'ds', 'DEOK_MW': 'y'}, inplace=True)
df_deok['ds'] = pd.to_datetime(df_deok['ds'], format='%Y-%m-%d %H:%M:%S')
out_dir = 'Deok_results'

### test prophet based without seasonality

In [ ]:
daily_demand_deok = df_deok.groupby(df_deok['ds'].dt.date)['y'].sum().reset_index()
date_start = pd.to_datetime('2013-10-01 00:00:00')
date_end = pd.to_datetime('2017-10-01 00:00:00')

results_prophet = []
i = 1

while True:

    print(f"fit model {date_end + pd.Timedelta(days=1)}")
    result = prophet_based.forecast_next_day_hourly(df_deok, date_start, date_end, daily_demand_deok, manual = False, daily = True)
    result['h_ahead'] = pd.to_datetime(result['ds']).dt.hour
    results_prophet.append(result)

    date_start += pd.Timedelta(days=1)
    date_end += pd.Timedelta(days=1)

    if i == 70:
        break
    else:
        i += 1

results_prophet = pd.concat(results_prophet, ignore_index=True)

results_prophet.to_csv(out_dir + '/results_prophet.csv', index=False)

### test prophet based with weekly seasonality

In [ ]:
daily_demand_deok = df_deok.groupby(df_deok['ds'].dt.date)['y'].sum().reset_index()
date_start = pd.to_datetime('2013-10-01 00:00:00')
date_end = pd.to_datetime('2017-10-01 00:00:00')

results_prophet_w = []
i = 1

while True:

    print(f"fit model {date_end + pd.Timedelta(days=1)}")
    result = prophet_based.forecast_next_day_hourly(df_deok, date_start, date_end, daily_demand_deok, manual = False, daily = True, weekly = True)
    result['h_ahead'] = pd.to_datetime(result['ds']).dt.hour
    results_prophet_w.append(result)

    date_start += pd.Timedelta(days=1)
    date_end += pd.Timedelta(days=1)

    if i == 70:
        break
    else:
        i += 1

results_prophet_w = pd.concat(results_prophet_w, ignore_index=True)

results_prophet_w.to_csv(out_dir + '/results_prophet_w.csv', index=False)

### test RNN+self convolution

In [ ]:
train_config = model_class.training_config(n_epochs = 30, device = torch.device("mps"))
results_aep_self_con = pd.DataFrame({'day': [], 'hour': [], 'y_hat': [], 'y':[]})
date_start = pd.to_datetime('2013-10-01 00:00:00')
date_end = pd.to_datetime('2017-10-01 00:00:00')

i = 1

while True:
    print(i)
    df_reduced = df_deok.loc[(df_deok['ds'] >= date_start) & (df_deok['ds'] < date_end)].copy()
    model = model_class.RNN_FeatureAttention()
    trainer = model_class.RNN_train_1(model, train_config)
    forcast, true = trainer(df_reduced)
    
    result = pd.DataFrame({'day': df_reduced['ds'].dt.date.max(), 'hour': list(range(24)), 'y_hat': forcast, 'y': true})

    results_aep_self_con = pd.concat([results_aep_self_con, result]).reset_index(drop=True)

    date_start += pd.Timedelta(days=1)
    date_end += pd.Timedelta(days=1)
    if i == 70:
        print('finish')
        break
    else:
        i += 1

results_aep_self_con.to_csv(out_dir + '/results_aep_self_con.csv', index=False)

### test RNN+self convolution + numerical weekday

In [ ]:
train_config = model_class.training_config(n_epochs = 30, device = torch.device("mps"))
results_aep_self_con_wn = pd.DataFrame({'day': [], 'hour': [], 'y_hat': [], 'y':[]})
date_start = pd.to_datetime('2013-10-01 00:00:00')
date_end = pd.to_datetime('2017-10-01 00:00:00')

i = 1

while True:
    print(i)
    df_reduced = df_deok.loc[(df_deok['ds'] >= date_start) & (df_deok['ds'] < date_end)].copy()
    model = model_class.RNN_FeatureAttention_wn()
    trainer = model_class.RNN_train_1(model, train_config)
    forcast, true = trainer(df_reduced)
    
    result = pd.DataFrame({'day': df_reduced['ds'].dt.date.max(), 'hour': list(range(24)), 'y_hat': forcast, 'y': true})

    results_aep_self_con_wn = pd.concat([results_aep_self_con_wn, result]).reset_index(drop=True)

    date_start += pd.Timedelta(days=1)
    date_end += pd.Timedelta(days=1)
    if i == 70:
        print('finish')
        break
    else:
        i += 1

results_aep_self_con_wn.to_csv(out_dir + '/results_aep_self_con_wn.csv', index=False)

### test RNN+self convolution + categorical weekday

In [ ]:
train_config = model_class.training_config(n_epochs = 30, device = torch.device("mps"))
results_aep_self_con_wc = pd.DataFrame({'day': [], 'hour': [], 'y_hat': [], 'y':[]})
date_start = pd.to_datetime('2013-10-01 00:00:00')
date_end = pd.to_datetime('2017-10-01 00:00:00')

i = 1

while True:
    print(i)
    df_reduced = df_deok.loc[(df_deok['ds'] >= date_start) & (df_deok['ds'] < date_end)].copy()
    model = model_class.RNN_FeatureAttention_wc()
    trainer = model_class.RNN_train_2(model, train_config)
    forcast, true = trainer(df_reduced)
    
    result = pd.DataFrame({'day': df_reduced['ds'].dt.date.max(), 'hour': list(range(24)), 'y_hat': forcast, 'y': true})

    results_aep_self_con_wc = pd.concat([results_aep_self_con_wc, result]).reset_index(drop=True)

    date_start += pd.Timedelta(days=1)
    date_end += pd.Timedelta(days=1)
    if i == 70:
        print('finish')
        break
    else:
        i += 1
results_aep_self_con_wc.to_csv(out_dir + '/results_aep_self_con_wc.csv', index=False)

### test RNN+self convolution + fourier vector (only weekly)

In [ ]:
train_config = model_class.training_config(n_epochs = 30, device = torch.device("mps"))
fourier_conf = model_class.fourier_config(mode="vector", K_weekly=3, K_monthly=0)
F_pairs = 2*(fourier_conf.K_weekly + fourier_conf.K_monthly)
if fourier_conf.mode == "vector":
    cont_dim = 1 + F_pairs
    fourier_dim = F_pairs
else:
    cont_dim = 1 + 24*F_pairs
    fourier_dim = F_pairs
            
results_aep_fourier_w = pd.DataFrame({'day': [], 'hour': [], 'y_hat': [], 'y':[]})
date_start = pd.to_datetime('2013-10-01 00:00:00')
date_end = pd.to_datetime('2017-10-01 00:00:00')

i = 1

while True:
    print(i)
    df_reduced = df_deok.loc[(df_deok['ds'] >= date_start) & (df_deok['ds'] < date_end)].copy()
    model = model_class.RNN_fourier(cont_dim=cont_dim, fourier_dim=fourier_dim,
                                    xf_mode=fourier_conf.mode, d_model=128, nhead=4, 
                                    activation="relu", learn_z0=True, dropout=0.0,
                                    H=24, use_gate=True, nonneg_U0=False)
    trainer = model_class.RNN_train_fourier(model, train_config, fourier_conf)
    forcast, true = trainer(df_reduced)
    
    result = pd.DataFrame({'day': df_reduced['ds'].dt.date.max(), 'hour': list(range(24)), 'y_hat': forcast, 'y': true})

    results_aep_fourier_w = pd.concat([results_aep_fourier_w, result]).reset_index(drop=True)

    date_start += pd.Timedelta(days=1)
    date_end += pd.Timedelta(days=1)
    if i == 70:
        print('finish')
        break
    else:
        i += 1

results_aep_fourier_w.to_csv(out_dir + '/results_aep_fourier_w.csv', index=False)

### test RNN+self convolution + fourier matrix (only weekly)

In [ ]:
train_config = model_class.training_config(n_epochs = 30, device = torch.device("mps"))
fourier_conf = model_class.fourier_config(mode="matrix", K_weekly=3, K_monthly=0)
F_pairs = 2*(fourier_conf.K_weekly + fourier_conf.K_monthly)
if fourier_conf.mode == "vector":
    cont_dim = 1 + F_pairs
    fourier_dim = F_pairs
else:
    cont_dim = 1 + 24*F_pairs
    fourier_dim = F_pairs
            
results_aep_fourier_w_matrix = pd.DataFrame({'day': [], 'hour': [], 'y_hat': [], 'y':[]})
date_start = pd.to_datetime('2013-10-01 00:00:00')
date_end = pd.to_datetime('2017-10-01 00:00:00')

i = 1

while True:
    print(i)
    df_reduced = df_deok.loc[(df_deok['ds'] >= date_start) & (df_deok['ds'] < date_end)].copy()
    model = model_class.RNN_fourier(cont_dim=cont_dim, fourier_dim=fourier_dim,
                                    xf_mode=fourier_conf.mode, d_model=128, nhead=4, 
                                    activation="relu", learn_z0=True, dropout=0.0,
                                    H=24, use_gate=True, nonneg_U0=False)
    trainer = model_class.RNN_train_fourier(model, train_config, fourier_conf)
    forcast, true = trainer(df_reduced)
    
    result = pd.DataFrame({'day': df_reduced['ds'].dt.date.max(), 'hour': list(range(24)), 'y_hat': forcast, 'y': true})

    results_aep_fourier_w_matrix = pd.concat([results_aep_fourier_w_matrix, result]).reset_index(drop=True)

    date_start += pd.Timedelta(days=1)
    date_end += pd.Timedelta(days=1)
    if i == 70:
        print('finish')
        break
    else:
        i += 1

results_aep_fourier_w_matrix.to_csv(out_dir + '/results_aep_fourier_w_matrix.csv', index=False)

### test RNN_based asym

In [ ]:
train_config = model_class.training_config(n_epochs = 30, device = torch.device("mps"))
results_aep_asy_con = pd.DataFrame({'day': [], 'hour': [], 'y_hat': [], 'y':[]})
date_start = pd.to_datetime('2013-10-01 00:00:00')
date_end = pd.to_datetime('2017-10-01 00:00:00')

i = 1

while True:
    print(i)
    df_reduced = df_deok.loc[(df_deok['ds'] >= date_start) & (df_deok['ds'] < date_end)].copy()
    model = model_class.RNN_asym_feature()
    trainer = model_class.RNN_train_1(model, train_config)
    forcast, true = trainer(df_reduced)

    result = pd.DataFrame({'day': df_reduced['ds'].dt.date.max(), 'hour': list(range(24)), 'y_hat': forcast, 'y': true})

    results_aep_asy_con = pd.concat([results_aep_asy_con, result]).reset_index(drop=True)

    date_start += pd.Timedelta(days=1)
    date_end += pd.Timedelta(days=1)
    if i == 70:
        print('finish')
        break
    else:
        i += 1

results_aep_asy_con.to_csv(out_dir + '/results_aep_asy_con.csv', index=False)

### Test RNN_based ARMA

In [ ]:
train_config = model_class.training_config(n_epochs = 30, device = torch.device("cpu"))
results_arma_asy_con = pd.DataFrame({'day': [], 'hour': [], 'y_hat': [], 'y':[]})
date_start = pd.to_datetime('2013-10-01 00:00:00')
date_end = pd.to_datetime('2017-10-01 00:00:00')

i = 1

while True:
    print(i)
    df_reduced = df_deok.loc[(df_deok['ds'] >= date_start) & (df_deok['ds'] < date_end)].copy()
    model = model_class.RNN_ARMA_Feature()
    trainer = model_class.RNN_train_1(model, train_config)
    forcast, true = trainer(df_reduced)
    result = pd.DataFrame({'day': df_reduced['ds'].dt.date.max(), 'hour': list(range(24)), 'y_hat': forcast, 'y': true})

    results_arma_asy_con = pd.concat([results_arma_asy_con, result]).reset_index(drop=True)

    date_start += pd.Timedelta(days=1)
    date_end += pd.Timedelta(days=1)
    if i == 70:
        print('finish')
        break
    else:
        i += 1

results_arma_asy_con.to_csv(out_dir + '/results_arma_asy_con.csv', index=False)